## [[Stack Overflow] Gathering a sequence of unknown length in dask #255](https://github.com/coiled/dask-community/issues/255)

In [1]:
from time import sleep
from distributed import Client, get_client, worker_client, Queue

In [2]:
def f(x):
    sleep(0.5)
    return (x - 1)**3

In [3]:
def derivative(x):
    sleep(1)
    return 3 * (x - 1)**2

In [4]:
def newton_optimization(x, fval, dfdx):
    if abs(fval) < 1e-10:
        return x
    with worker_client() as client:
        x = x - fval / dfdx
        # client = get_client()
        fval = client.submit(f, x)
        dfdx = client.submit(derivative, x)
        next_step = client.submit(newton_optimization, x, fval, dfdx)
        queue.put(next_step)
    return x

In [5]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:54008,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:54022,Total threads: 2
Dashboard: http://127.0.0.1:54024/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:54014,


In [6]:
queue = Queue()

In [7]:
task = client.submit(newton_optimization, 0, 1, 3)

In [13]:
task.result()

-0.3333333333333333

In [ ]:
while True:
    future = queue.get()
    print(future.result())

In [ ]:
client.shutdown()